Librerías y conexión a la base de datos

Se importan las librerías para las bases de datos

In [129]:
import pandas as pd
import numpy as np
#import sqlite3
from openpyxl import Workbook
#from dbi import connect
import pyodbc
#from pyodbc import connect as pyodbc_connect
#from sqlite3 import connect as sqlite3_connect
from scipy import stats
from scipy import special
from scipy.stats import gmean

In [130]:
import funciones as f

In [131]:
anio = int(input('Ingrese el año: '))
mes = int(input('Ingrese el mes: '))

#anio = 2024
#mes = 3

In [132]:
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [133]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [134]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    sql_command = f"EXEC [dbo].[sp_get_calculos_precios_recolectados_mes] {anio}, {mes}"
    cursor.execute(sql_command)
    boletas = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])
    
    sql_command01 = f"EXEC sp_get_indice_grupo {anio}, {mes}"
    cursor.execute(sql_command01)
    ponderaciones = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [135]:
#boletas

___________________________________________________________________________________________

In [136]:
participacion = pd.DataFrame({
        'region': [1, 2, 3, 4, 5, 6, 7, 8],
        'participacion': [13.472203, 12.046949, 12.486967, 12.483411, 12.674423, 12.415842, 12.190962, 12.229243]
    })
participacion['region'] = participacion['region'].astype('int64')

In [137]:
bol = boletas[['region', 'codigo_articulo', 'articulo', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']].copy()
bol.insert(1, 'cod_prod', np.where(bol['codigo_articulo'].str.len() > 8, 
                                   bol['codigo_articulo'].str.slice(0, -2), 
                                   bol['codigo_articulo'].str.slice(0, -1)))
bol.loc[:, 'cod_prod'] = bol['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
bol['cantidad_anterior'] = pd.to_numeric(bol['cantidad_anterior'], errors='coerce')
bol['cantidad_actual'] = pd.to_numeric(bol['cantidad_actual'], errors='coerce')
bol['precio_anterior'] = pd.to_numeric(bol['precio_anterior'], errors='coerce')
bol['precio_actual'] = pd.to_numeric(bol['precio_actual'], errors='coerce')
bol.loc[bol['cantidad_anterior'] == 0 | bol['cantidad_anterior'].isnull(), 'cantidad_anterior'] = bol['cantidad_actual']
bol.loc[bol['precio_anterior'] == 0 | bol['precio_anterior'].isnull(), 'precio_anterior'] = bol['precio_actual']

#bol['cantidad_actual'] = bol['cantidad_anterior']
#bol['precio_actual'] = bol['precio_anterior']

bol

,region,cod_prod,codigo_articulo,articulo,cantidad_anterior,cantidad_actual,precio_anterior,precio_actual,nt_tipo
0,1,0540301,5403012,Olla y cacerola de 1 a 2lt,1.0,1.0,145.00,145.00,0
1,1,0312306,3123061,Vestidos formales con mangas de largo hasta la...,1.0,1.0,250.00,250.00,0
2,1,0721101,7211011,Llantas para automóvil 185/65 R14,1.0,1.0,375.00,375.00,0
3,1,0411101,4111013,Alquiler de apartamento (con o sin servicios),1.0,1.0,1700.00,1700.00,0
4,1,0321105,3211051,Sandalias para hombre,1.0,1.0,279.99,279.99,0
...,...,...,...,...,...,...,...,...,...
76089,8,1111208,11112083,Piezas de pollo individual o entero (sin acomp...,1.0,1.0,18.00,18.00,0
76090,8,1111208,11112084,Ensalada de verduras preparadas,1.0,1.0,45.00,45.00,0
76091,8,1312008,13120081,Paquetes de toallas húmedas para bebés de 60 a...,1.0,1.0,10.00,10.00,0
76092,8,0611120,6111202,Trimetoprim+sulfametoxazol suspensión 200mg/40...,100.0,100.0,35.00,35.00,0


In [138]:
# Se obtienen las ponderaciones de cada produto por cada region y los indices anteriores
pon_prod = f.pondcat('Producto', ponderaciones).rename(columns={'grupo_codigo':'cod_prod'})
pon_prod['cod_prod'] = pon_prod['cod_prod'].apply(lambda x: '0' + x if len(str(x)) == 6 else x)
pon_prod['ponderacion_region'] = pd.to_numeric(pon_prod['ponderacion_region'], errors='coerce')
pon_prod['indice_anterior'] = pd.to_numeric(pon_prod['indice_anterior'], errors='coerce')
#pon_prod

In [139]:
#pon_prod['ponderacion_region'].dtype
#pon_prod['ponderacion_region'].isnull().sum()

Calculo del IPC

In [140]:
# Se calcula la variacion de las variedades
variacion = f.variacion(bol)
variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)
variacion = variacion[['region', 'cod_prod', 'codigo_articulo', 'articulo', 'var_prod']]
#variacion['var_prod'].dtype
#variacion

In [141]:
variacion.to_excel('variacionpy.xlsx')

In [142]:
#relativo de precios de los productos
relativo = (variacion.groupby(['region', 'cod_prod'])
            .agg(relativo = ('var_prod', lambda x: gmean(x)))
            .reset_index())
relativo = pd.merge(relativo, pon_prod[['region', 'cod_prod', 'grupo_nombre', 'ponderacion_region', 'indice_anterior']],
         on=['region', 'cod_prod'], how='left')
relativo
#relativo['ponderacion_region'].dtype

#relativo[relativo['ponderacion_region'] == 0]

,region,cod_prod,relativo,grupo_nombre,ponderacion_region,indice_anterior
0,1,0111101,1.008833,Arroz,0.399291,105.375700
1,1,0111102,1.017687,Maíz,0.130256,95.976221
2,1,0111201,1.004664,Harina de trigo,0.072427,97.930513
3,1,0111202,1.009924,Harina de maíz,0.039212,103.073844
4,1,0111203,1.004116,Harina para atoles,0.066483,102.158495
...,...,...,...,...,...,...
3199,8,1329101,1.000000,Sombrillas y paraguas,0.050085,100.000000
3200,8,1329102,1.000000,Ataúdes,0.065264,100.000000
3201,8,1390901,1.000000,Servicios legales y financieros,0.194022,100.000000
3202,8,1390902,1.000000,Servicios funerarios,0.037005,100.000000


In [143]:
#nulos = relativo['ponderacion_region'].isnull()
#relativo[nulos]
relativo.to_excel('relativopy.xlsx')

In [144]:
#promedio ponderado
prom_pon = relativo
prom_pon['var_ac'] = prom_pon['relativo']*prom_pon['ponderacion_region']
prom_pon = prom_pon[prom_pon['var_ac'] != 0]
## lo unico que se me ocurre es hacer otros dataframes agrupados, uno para la suma de la var ac, otro para la suma de las ponderaciones y asi y luego unirlos

#prom_pon['sum_var_ac'] = prom_pon['var_ac'].sum()
prom_pon01 = prom_pon.groupby(['region', 'cod_prod', 'grupo_nombre']).agg(
    sum_var_ac=('var_ac', 'sum'),
    sum_ponderacion=('ponderacion_region', 'sum')
    ).reset_index()
prom_pon = prom_pon.merge(prom_pon01, on=['region', 'cod_prod', 'grupo_nombre'], how='left')
prom_pon['prom_pon'] = prom_pon['sum_var_ac']/prom_pon['sum_ponderacion']

#prom_pon
#prom_pon01
#prom_pon[prom_pon['var_ac'] == 0]
#prom_pon01[prom_pon01['sum_var_ac'] == 0]

In [145]:
prom_pon.to_excel('promponpy.xlsx')

Índice de precios por región

In [146]:
#índice productos
prom_pon['ind_prod'] = prom_pon['indice_anterior']*prom_pon['prom_pon']
ind_prod = f.cod_grupos(prom_pon).dropna()

ind_prod01 = ind_prod[['region', 'cod_prod', 'ind_prod']].copy()
#dist100['ind_prod'].dtype

In [147]:
ind_prod01.to_excel('indprodpy.xlsx')

In [148]:
#indice subclase
f.indices(ind_prod,'cod_subclase')

,region,grupo_codigo,ind
0,1,01111,104.182995
1,1,01112,101.208693
2,1,01113,98.935957
3,1,01114,100.454661
4,1,01115,97.875167
...,...,...,...
1232,8,13131,103.270931
1233,8,13132,100.000000
1234,8,13211,100.260409
1235,8,13291,100.000000


In [149]:
#indice clase
f.indices(ind_prod,'cod_clase')

,region,grupo_codigo,ind
0,1,0111,99.423780
1,1,0112,99.928257
2,1,0113,100.965603
3,1,0114,99.946238
4,1,0115,102.209594
...,...,...,...
691,8,1312,101.549508
692,8,1313,102.646954
693,8,1321,100.260409
694,8,1329,100.000000


In [150]:
#indice grupo
f.indices(ind_prod,'cod_grupo')

,region,grupo_codigo,ind
0,1,011,101.045403
1,1,012,99.988562
2,1,013,100.000000
3,1,021,98.707168
4,1,023,102.105299
...,...,...,...
327,8,111,100.231757
328,8,112,105.204840
329,8,131,101.923466
330,8,132,100.190945


In [151]:
#indice division
f.indices(ind_prod,'cod_div')

,region,grupo_codigo,ind
0,1,01,100.957174
1,1,02,99.640217
2,1,03,100.657251
3,1,04,100.234966
4,1,05,99.824547
...,...,...,...
95,8,08,100.332496
96,8,09,99.733194
97,8,10,100.260316
98,8,11,100.260405


Índice a nivel republica

In [152]:
#indice productos
f.indicerep(ind_prod,'cod_prod')

,cod_prod,ind_prod_rep
0,0111101,103.752231
1,0111102,97.070949
2,0111201,97.161032
3,0111202,101.099641
4,0111203,100.307632
...,...,...
432,1390901,100.000000
433,1390902,101.135848
434,1390903,100.000000
435,1390904,100.000000


In [153]:
#indice subclase
f.indicerep(ind_prod,'cod_subclase')

,cod_subclase,ind_prod_rep
0,01111,99.739862
1,01112,100.274987
2,01113,98.365902
3,01114,100.477850
4,01115,99.059702
...,...,...
163,13131,101.622114
164,13132,100.372539
165,13211,100.376028
166,13291,99.970660


In [154]:
#indice clase
f.indicerep(ind_prod,'cod_clase')

,cod_clase,ind_prod_rep
0,0111,98.744446
1,0112,101.579117
2,0113,97.633052
3,0114,96.871612
4,0115,102.332830
...,...,...
91,1312,99.720645
92,1313,101.489115
93,1321,100.376028
94,1329,99.970660


In [155]:
#indice grupo
f.indicerep(ind_prod,'cod_grupo')

,cod_grupo,ind_prod_rep
0,011,101.454678
1,012,101.024885
2,013,100.495584
3,021,99.315956
4,023,100.117266
5,031,100.518902
6,032,100.560983
7,041,100.527878
8,043,100.004090
9,044,104.389411


In [156]:
#indice division
f.indicerep(ind_prod,'cod_div')

,cod_div,ind_prod_rep
0,01,101.403547
1,02,99.478681
2,03,100.531258
3,04,99.677828
4,05,100.224255
5,06,100.512864
6,07,102.892683
7,08,100.454182
8,09,100.326572
9,10,102.598954


Indice general

In [157]:
#indice producos a nivel nacional
def indicenac(df):
    indice = df.groupby('region').apply(lambda x: pd.Series({
        'sum1': (x['ponderacion_region'] * x['ind_prod']).sum(),
        'sum2': x['ponderacion_region'].sum()
    })).reset_index()

    indice['ind_prod_rep'] = indice['sum1'] / indice['sum2']
    indice = indice[['region', 'ind_prod_rep']]

    return indice



In [158]:
indicenac(ind_prod)

,region,ind_prod_rep
0,1,100.955875
1,2,101.439635
2,3,100.828811
3,4,101.146369
4,5,100.685210
5,6,100.937398
6,7,101.143585
7,8,100.992566


In [159]:
def calculate_replicated_indicator(df):
    sum1 = (df['ponderacion_region'] * df['ind_prod']).sum()
    sum2 = df['ponderacion_region'].sum()

    result = pd.DataFrame({
        'sum1': [sum1],
        'sum2': [sum2]
    })

    result['ind_rep'] = result['sum1'] / result['sum2']

    return result

In [160]:
calculate_replicated_indicator(ind_prod)

,sum1,sum2,ind_rep
0,75243.144547,744.871243,101.014968
